In [1]:
from src.models.linreg import LinReg

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

In [2]:
np.random.seed(69)
x = np.linspace(0, 5, 50)
z = np.linspace(3, 8, 50)
y = 1 + 3 * x + 5*z + np.random.normal(0, 2, 50)
data = pd.DataFrame({'outcome': y,
                     'independent1': x,
                     'independent2': z})

In [3]:
linreg = LinReg(df=data, outcome="outcome", independent=["~independent2"])

x = data['independent1']
y = data['outcome']
x = sm.add_constant(x)
sm = sm.OLS(y, x).fit()

['~independent2']
using ~ operator
['~independent2']


In [4]:
linreg.independent_vars

['independent1']

In [5]:
sm.params

const           15.885937
independent1     7.832811
dtype: float64

In [6]:
linreg.summary(content_type='simple')

In [5]:
string = '~1'

'~' in string

True